In [ ]:
import asyncio
import nest_asyncio
import os, sys
import datetime
from collections import Counter

import warnings
nest_asyncio.apply()

warnings.simplefilter("error", RuntimeWarning)  # 경고를 예외로 변환

home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "GitHub", "Thunder", "Binance"))

import Workspace.Utils.TradingUtils as tr_utils# import 
from Workspace.Services.PublicData.Receiver.FuturesMarketWebsocket import (
    FuturesMarketWebsocket as futures_mk_ws,
)
import Workspace.Utils.BaseUtils as base_utils
import SystemConfig
# import Workspace.Utils.BaseUtils as base_utils


symbols = SystemConfig.Streaming.symbols
stream_1 = "depth"
stream_2 = "trade"

ins_st1 = futures_mk_ws(symbols)
ins_st2 = futures_mk_ws(symbols)
async def main(minute:int):
    print(f"  ⏳ make the program wait")
    # await base_utils.sleep_next_minute(1)
    st1_result = []
    st2_result = []
    
    start_date = datetime.datetime.now()
    now_minute = int(start_date.minute)
    target_minute = int(start_date.minute) + minute
    await ins_st1.open_stream_connection(stream_1)
    print(f"  🔗 '{stream_1}' Websocket connect")
    await ins_st2.open_stream_connection(stream_2)
    print(f"  🔗 '{stream_2}' Websocket connect")
    i = 1
    print(f"  🚀 start receiving start")
    bids_prices = []
    asks_prices = []
    while now_minute < target_minute:
        # base_utils.std_print(f"        🏃🏃🏻‍♀️🏃🏽‍♂️💨 {i}회 진행")
        st1_message = await ins_st1.receive_message()
        st2_message = await ins_st2.receive_message()
        i +=1
        
        symbol_st1 = st1_message["data"]["s"]
        symbol_st2 = st2_message["data"]["s"]


        if symbol_st1 == "BTCUSDT":
            data = st1_message["data"]
            bids = data.get("b", [])  # 매수 주문 업데이트
            asks = data.get("a", [])  # 매도 주문 업데이트

            for bid in bids:
                price, quantity = bid
                if float(quantity) == 0:
                    convert_to_price = int(float(price))
                    bids_prices.append(convert_to_price)

            for ask in asks:
                price, quantity = ask
                if float(quantity) == 0:
                    convert_to_price = int(float(price))
                    asks_prices.append(convert_to_price)
    
        new_date = datetime.datetime.now()
        now_minute = int(new_date.minute)
        

    print(f"\n  ✅ 데이터 수신 완료")
    await ins_st1.close_connection()
    await ins_st2.close_connection()
    return bids_prices, asks_prices

message = asyncio.run(main(2))

bid, ask = message

data = ['apple', 'banana', 'apple', 'orange', 'banana', 'apple']
bid_count = Counter(bid)
bid_count.
bid_ranking = bid_count.most_common() 

ask_count = Counter(ask)
ask_ranking = ask_count.most_common() 

print(bid_ranking)
print(ask_ranking)

In [ ]:
ranking_asc = sorted(bid_ranking.items(), key=lambda x: x[1])

In [1]:
import nest_asyncio
nest_asyncio.apply()  # 주피터 노트북에서 asyncio 사용 가능하게 설정

import asyncio
import json
import websockets
from collections import deque

SYMBOL = "btcusdt"
URL = f"wss://fstream.binance.com/ws/{SYMBOL}@depth"

# 누적 델타 볼륨 계산을 위한 큐
delta_volume_queue = deque(maxlen=300)  # 최근 50개의 델타 값 저장

async def process_order_book():
    async with websockets.connect(URL, ping_interval=10) as ws:
        while True:
            data = await ws.recv()
            order_book = json.loads(data)
            
            bids = order_book["b"][:5]  # 상위 5개 매수 호가
            asks = order_book["a"][:5]  # 상위 5개 매도 호가

            # 최우선 매수(Bid) 및 매도(Ask) 가격
            best_bid = float(bids[0][0])
            best_ask = float(asks[0][0])
            spread = best_ask - best_bid  # 스프레드 계산

            # 매수·매도 물량 총합 (상위 5개 호가)
            total_bid_volume = sum(float(bid[1]) for bid in bids)
            total_ask_volume = sum(float(ask[1]) for ask in asks)

            # 오더북 불균형 계산
            imbalance = (total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)

            # 대량 주문 감지 (100 BTC 이상 주문 존재 여부 확인)
            large_bid_orders = [bid for bid in bids if float(bid[1]) > 100]
            large_ask_orders = [ask for ask in asks if float(ask[1]) > 100]

            large_bid_detected = len(large_bid_orders) > 0
            large_ask_detected = len(large_ask_orders) > 0

            # 누적 델타 볼륨 업데이트
            delta_volume = total_bid_volume - total_ask_volume
            delta_volume_queue.append(delta_volume)
            avg_delta_volume = sum(delta_volume_queue) / len(delta_volume_queue)

            # 결과 출력
            print(f"\n🔹 Best Bid: {best_bid}, Best Ask: {best_ask}, Spread: {spread:.5f}")
            print(f"📊 Order Book Imbalance: {imbalance:.3f}")
            print(f"📈 Large Buy Orders Detected: {large_bid_detected}, 📉 Large Sell Orders Detected: {large_ask_detected}")
            print(f"📊 Cumulative Delta Volume (Last 50): {avg_delta_volume:.2f}")

            # 0.5초 대기 후 반복
            await asyncio.sleep(0.5)

asyncio.run(process_order_book())



🔹 Best Bid: 1000.0, Best Ask: 81853.8, Spread: 80853.80000
📊 Order Book Imbalance: 0.994
📈 Large Buy Orders Detected: True, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta Volume (Last 50): 372.96

🔹 Best Bid: 1000.0, Best Ask: 81853.8, Spread: 80853.80000
📊 Order Book Imbalance: 0.990
📈 Large Buy Orders Detected: True, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta Volume (Last 50): 378.44

🔹 Best Bid: 1000.0, Best Ask: 81853.8, Spread: 80853.80000
📊 Order Book Imbalance: 1.000
📈 Large Buy Orders Detected: True, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta Volume (Last 50): 376.53

🔹 Best Bid: 1000.0, Best Ask: 81853.8, Spread: 80853.80000
📊 Order Book Imbalance: 1.000
📈 Large Buy Orders Detected: True, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta Volume (Last 50): 378.09

🔹 Best Bid: 1000.0, Best Ask: 81859.9, Spread: 80859.90000
📊 Order Book Imbalance: 0.973
📈 Large Buy Orders Detected: True, 📉 Large Sell Orders Detected: False
📊 Cumulative Delta 

ConnectionClosedError: sent 1011 (internal error) keepalive ping timeout; no close frame received

In [5]:
import nest_asyncio
import asyncio
import json
import websockets
import numpy as np
from collections import deque

nest_asyncio.apply()

SYMBOL = "btcusdt"
URL = f"wss://fstream.binance.com/ws/{SYMBOL}@depth10@100ms"

# ✅ 최근 3000개의 데이터만 유지 (누적 분석을 위해 deque 사용)
depth_data = deque(maxlen=3000)

async def process_order_book():
    async with websockets.connect(URL, ping_interval=10, ping_timeout=30) as ws:
        while True:
            data = await ws.recv()
            order_book = json.loads(data)

            # ✅ 매수·매도 호가를 np.array 변환 (오버헤드 최소화)
            bid_prices, bid_qtys = np.array(order_book["b"], dtype=float).T
            ask_prices, ask_qtys = np.array(order_book["a"], dtype=float).T

            # ✅ 기존 데이터에서 이전 주문 정보 가져오기
            if depth_data:
                prev_bids = depth_data[-1]["bids"]
                prev_asks = depth_data[-1]["asks"]
            else:
                prev_bids = (np.array([]), np.array([]))
                prev_asks = (np.array([]), np.array([]))

            # ✅ 신규 주문 및 취소 주문 계산 함수
            def calc_order_changes(prev_orders, new_orders):
                prev_prices, prev_qtys = prev_orders
                new_prices, new_qtys = new_orders

                prev_dict = {p: q for p, q in zip(prev_prices, prev_qtys)}
                new_dict = {p: q for p, q in zip(new_prices, new_qtys)}

                # 신규 주문: 새로운 가격이 생기거나, 기존 가격에서 수량 증가
                new_orders = {p: q for p, q in new_dict.items() if p not in prev_dict or new_dict[p] > prev_dict[p]}
                new_order_value = sum(p * q for p, q in new_orders.items())

                # 취소 주문: 기존에 있었지만 수량이 0이 되었거나 감소한 경우
                canceled_orders = {p: q for p, q in prev_dict.items() if p not in new_dict or new_dict[p] < prev_dict[p]}
                canceled_order_value = sum(p * q for p, q in canceled_orders.items())

                return new_order_value, canceled_order_value

            # ✅ 신규 주문 및 취소 주문 금액 계산
            bid_new_value, bid_canceled_value = calc_order_changes(prev_bids, (bid_prices, bid_qtys))
            ask_new_value, ask_canceled_value = calc_order_changes(prev_asks, (ask_prices, ask_qtys))

            # ✅ depth_data에 최신 데이터 추가 (pop 사용하지 않음, list(deque) 활용)
            depth_data.append({"bids": (bid_prices, bid_qtys), "asks": (ask_prices, ask_qtys)})

            # ✅ 누적된 데이터 활용 (list(deque) 변환)
            all_data = list(depth_data)

            # ✅ 전체 누적 데이터에서 총 주문 변화량 계산
            total_bid_new = sum(calc_order_changes(all_data[i-1]["bids"], all_data[i]["bids"])[0] for i in range(1, len(all_data)))
            total_bid_canceled = sum(calc_order_changes(all_data[i-1]["bids"], all_data[i]["bids"])[1] for i in range(1, len(all_data)))
            total_ask_new = sum(calc_order_changes(all_data[i-1]["asks"], all_data[i]["asks"])[0] for i in range(1, len(all_data)))
            total_ask_canceled = sum(calc_order_changes(all_data[i-1]["asks"], all_data[i]["asks"])[1] for i in range(1, len(all_data)))

            # ✅ 실시간 및 누적 데이터 출력
            print("\n📌 [100ms 업데이트] Futures Depth 변경 내역")
            print(f"  🟢 실시간 신규 매수 주문: {bid_new_value:.2f} USDT")
            print(f"  🔴 실시간 취소 매수 주문: {bid_canceled_value:.2f} USDT")
            print(f"  🟢 실시간 신규 매도 주문: {ask_new_value:.2f} USDT")
            print(f"  🔴 실시간 취소 매도 주문: {ask_canceled_value:.2f} USDT")

            print("\n📊 [누적 데이터] (최근 3000개 기준)")
            print(f"  🟢 누적 신규 매수 주문 총액: {total_bid_new:,.2f} USDT")
            print(f"  🔴 누적 취소 매수 주문 총액: {total_bid_canceled:,.2f} USDT")
            print(f"  🟢 누적 신규 매도 주문 총액: {total_ask_new:,.2f} USDT")
            print(f"  🔴 누적 취소 매도 주문 총액: {total_ask_canceled:,.2f} USDT")

            await asyncio.sleep(0.1)  # ✅ 100ms 간격 유지

asyncio.run(process_order_book())



📌 s 업데이트] Futures Depth 변경 내역
  🟢 실시간 신규 매수 주문: 740495.81 USDT
  🔴 실시간 취소 매수 주문: 0.00 USDT
  🟢 실시간 신규 매도 주문: 223077.06 USDT
  🔴 실시간 취소 매도 주문: 0.00 USDT

📊 [누적 데이터] (최근 3000개 기준)
  🟢 누적 신규 매수 주문 총액: 0.00 USDT
  🔴 누적 취소 매수 주문 총액: 0.00 USDT
  🟢 누적 신규 매도 주문 총액: 0.00 USDT
  🔴 누적 취소 매도 주문 총액: 0.00 USDT

📌 s 업데이트] Futures Depth 변경 내역
  🟢 실시간 신규 매수 주문: 668734.82 USDT
  🔴 실시간 취소 매수 주문: 0.00 USDT
  🟢 실시간 신규 매도 주문: 0.00 USDT
  🔴 실시간 취소 매도 주문: 164108.41 USDT

📊 [누적 데이터] (최근 3000개 기준)
  🟢 누적 신규 매수 주문 총액: 668,734.82 USDT
  🔴 누적 취소 매수 주문 총액: 0.00 USDT
  🟢 누적 신규 매도 주문 총액: 0.00 USDT
  🔴 누적 취소 매도 주문 총액: 164,108.41 USDT

📌 s 업데이트] Futures Depth 변경 내역
  🟢 실시간 신규 매수 주문: 0.00 USDT
  🔴 실시간 취소 매수 주문: 0.00 USDT
  🟢 실시간 신규 매도 주문: 185759.90 USDT
  🔴 실시간 취소 매도 주문: 0.00 USDT

📊 [누적 데이터] (최근 3000개 기준)
  🟢 누적 신규 매수 주문 총액: 668,734.82 USDT
  🔴 누적 취소 매수 주문 총액: 0.00 USDT
  🟢 누적 신규 매도 주문 총액: 185,759.90 USDT
  🔴 누적 취소 매도 주문 총액: 164,108.41 USDT

📌 s 업데이트] Futures Depth 변경 내역
  🟢 실시간 신규 매수 주문: 10579.55 USDT
  🔴 실시간 취소 매수 주

ConnectionClosedError: sent 1011 (internal error) keepalive ping timeout; no close frame received

In [3]:
prev_prices = [62000, 61950, 61800]
prev_qtys = [0.5, 1.2, 0.8]

# 가격을 Key, 수량을 Value로 딕셔너리 생성
prev_dict = {p: q for p, q in zip(prev_prices, prev_qtys)}

print(prev_dict)


{62000: 0.5, 61950: 1.2, 61800: 0.8}


In [ ]:
symbols = ['BTCUSDT', 'XRPUSDT', 'ADAUSDT',]
intervals = ["1m", "3m", "5m", "15m", "1d", "1y"]

data = {p: q for p, q in zip(symbols, intervals)}

In [5]:
data

{'BTCUSDT': '1m', 'XRPUSDT': '3m', 'ADAUSDT': '5m'}